In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import getenv
from binance.client import Client
from binance.websockets import BinanceSocketManager
from twisted.internet import reactor

In [ ]:
# Statistical Arbitrage - Pair Trading Strategy

In [2]:
# client login to binance.us
client = Client(getenv('binance_api'), getenv('binance_secret'))

In [6]:
# valid intervals - 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
# get timestamp of earliest date data is available
ticker = 'BTCUSDT'
interval = '1d'
timestamp = client._get_earliest_valid_timestamp(ticker, interval)

1502928000000

In [7]:
# request historical candle (or klines) data
bars = client.get_historical_klines(ticker, interval, timestamp)

In [11]:
# delete unwanted data - just keep date, open, high, low, close
for line in bars:
    del line[5:]

In [14]:
btc_df = pd.DataFrame(bars, columns=['date', 'open', 'high', 'low', 'close'])
btc_df.set_index('date', inplace=True)
btc_df.index = pd.to_datetime(btc_df.index, unit='ms')
print(btc_df.tail())

                open      high       low     close
date                                              
2021-02-07  39181.01  39700.00  37351.00  38795.69
2021-02-08  38795.69  46794.45  37988.89  46374.87
2021-02-09  46374.86  48142.19  44961.09  46420.42
2021-02-10  46420.42  47310.00  43727.00  44807.58
2021-02-11  44807.58  48399.00  43994.02  47500.00


In [26]:
interval = '1d'

def get_tickers():
    ticker = []
    tickers = client.get_all_tickers()
    for item in tickers:
        ticker.append(item['symbol'])
    return ticker

def get_historical(ticker, interval):
    timestamp = client._get_earliest_valid_timestamp(ticker, interval)
    bars = client.get_historical_klines(ticker, interval, timestamp)
    for line in bars:
        del line[5:]
    date = []
    open_price = []
    high_price = []
    low_price = []
    close_price = []
    for item in bars:
        date.append(item['date'])
        open_price.append(item['open'])
        high_price.append(item['high'])
        low_price.append(item['low'])
        close_price.append(item['close'])
    
    date = pd.Series(date)
    date = pd.to_datetime(date, unit='ms')
    open_price = pd.Series(open_price)
    high_price = pd.Series(high_price)
    low_price = pd.Series(low_price)
    close_price = pd.Series(close_price)
    series = [open_price.rename("Open"), high_price.rename("High"), low_price.rename("Low"), close_price.rename("Close"), date.rename("Dates")]
    df = pd.concat(series, axis=1)
    df.set_index('date', inplace=True)
    return df

In [27]:
get_historical(df)

NameError: name 'df' is not defined